In [2]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

import tkinter as tk
from tkinter import messagebox

def login():
    # Load HAAR face classifier
    face_classifier = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')

    # Load functions
    def face_extractor(img):
        # Function detects faces and returns the cropped face
        # If no face detected, it returns the input image
        
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        
        if faces is ():
            return None
        
        # Crop all faces found
        for (x,y,w,h) in faces:
            cropped_face = img[y:y+h, x:x+w]

        return cropped_face

    # Initialize Webcam
    cap = cv2.VideoCapture(0)
    count = 0
# Collect 100 samples of your face from webcam input
    while True:

        ret, frame = cap.read()
        if face_extractor(frame) is not None:
            count += 1
            face = cv2.resize(face_extractor(frame), (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            # Save file in specified directory with unique name
            file_name_path = './faces/user/' + str(count) + '.jpg'
            cv2.imwrite(file_name_path, face)

            # Put count on images and display live count
            cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Cropper', face)
            
        else:
            print("Face not found")
            pass

        if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
            break
            
        cap.release()
        cv2.destroyAllWindows()      
        print("Collecting Samples Complete")







    # Get the training data we previously made
    data_path = './faces/user/'
    onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

    # Create arrays for training data and labels
    Training_Data, Labels = [], []

    # Open training images in our datapath
    # Create a numpy array for training data
    for i, files in enumerate(onlyfiles):
        image_path = data_path + onlyfiles[i]
        images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        Training_Data.append(np.asarray(images, dtype=np.uint8))
        Labels.append(i)

    # Create a numpy array for both training data and labels
    Labels = np.asarray(Labels, dtype=np.int32)

    # Initialize facial recognizer
    model = cv2.face.LBPHFaceRecognizer_create()
    # NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()

    # Let's train our model 
    model.train(np.asarray(Training_Data), np.asarray(Labels))
    print("Model trained sucessefully")


def register():
    face_classifier = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')

    def face_detector(img, size=0.5):
        
        # Convert image to grayscale
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        if faces is ():
            return img, []
        
        for (x,y,w,h) in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
            roi = img[y:y+h, x:x+w]
            roi = cv2.resize(roi, (200, 200))
        return img, roi


    # Open Webcam
    cap = cv2.VideoCapture(0)

    while True:

        ret, frame = cap.read()
        
        image, face = face_detector(frame)
        
        try:
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            # Pass face to prediction model
            # "results" comprises of a tuple containing the label and the confidence value
            results = model.predict(face)
            
            if results[1] < 500:
                confidence = int( 100 * (1 - (results[1])/400) )
                display_string = str(confidence) + '% Confident it is User'
                
            cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
            
            if confidence > 75:
                cv2.putText(image, "Unlocked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
                cv2.imshow('Face Recognition', image )
            else:
                cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
                cv2.imshow('Face Recognition', image )

        except:
            cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )
            pass
            
        if cv2.waitKey(1) == 13: #13 is the Enter Key
            break
            
    cap.release()
    cv2.destroyAllWindows()     

#Tkinter
root = tk.Tk()
root.title("Face Login Example")
#Frames
loginFrame=tk.Frame(root)
regFrame=tk.Frame(root)
userMenuFrame = tk.Frame(root)

#Define Frame List
frameList=[loginFrame,regFrame,userMenuFrame]
#Configure all Frames
for frame in frameList:
	frame.grid(row=0,column=0, sticky='news')
	frame.configure(bg='white')
	
def raiseFrame(frame):
	frame.tkraise()

def regFrameRaiseFrame():
	raiseFrame(regFrame)
def logFrameRaiseFrame():
	raiseFrame(loginFrame)
#Tkinter Vars
#Stores user's name when registering
name = tk.StringVar()
#Stores user's name when they have logged in
loggedInUser = tk.StringVar()


tk.Label(loginFrame,text="Face Recognition",font=("Courier", 60),bg="white").grid(row=1,column=1,columnspan=5)
loginButton = tk.Button(loginFrame,text="Login",bg="white",font=("Arial", 30),command=login)
loginButton.grid(row=2,column=5)
regButton = tk.Button(loginFrame,text="Register",command=regFrameRaiseFrame,bg="white",font=("Arial", 30))
regButton.grid(row=2,column=1)

tk.Label(regFrame,text="Register",font=("Courier",60),bg="white").grid(row=1,column=1,columnspan=5)
tk.Label(regFrame,text="Name: ",font=("Arial",30),bg="white").grid(row=2,column=1)
nameEntry=tk.Entry(regFrame,textvariable=name,font=("Arial",30)).grid(row=2,column=2)

registerButton = tk.Button(regFrame,text="Register",command=register,bg="white",font=("Arial", 30))
registerButton.grid(row=3,column=2)

tk.Label(userMenuFrame,text="Hello, ",font=("Courier",60),bg="white").grid(row=1,column=1)
tk.Label(userMenuFrame,textvariable=loggedInUser,font=("Courier",60),bg="white",fg="red").grid(row=1,column=2)
tk.Button(userMenuFrame,text="Back",font=("Arial", 30),command=logFrameRaiseFrame).grid(row=2,column=1)



<>:21: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:102: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:21: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:102: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-2-1812607bab9a>:21: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():
<ipython-input-2-1812607bab9a>:102: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


In [ ]:
import cv2
import numpy as np


face_classifier = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        results = model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence > 75:
            cv2.putText(image, "Unlocked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
        else:
            cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
cap.release()
cv2.destroyAllWindows()

In [3]:

import tkinter as tk
from tkinter import messagebox
#Tkinter
root = tk.Tk()
root.title("Face Login Example")
#Frames
loginFrame=tk.Frame(root)
regFrame=tk.Frame(root)
userMenuFrame = tk.Frame(root)

#Define Frame List
frameList=[loginFrame,regFrame,userMenuFrame]
#Configure all Frames
for frame in frameList:
	frame.grid(row=0,column=0, sticky='news')
	frame.configure(bg='white')
	
def raiseFrame(frame):
	frame.tkraise()

def regFrameRaiseFrame():
	raiseFrame(regFrame)
def logFrameRaiseFrame():
	raiseFrame(loginFrame)
#Tkinter Vars
#Stores user's name when registering
name = tk.StringVar()
#Stores user's name when they have logged in
loggedInUser = tk.StringVar()


tk.Label(loginFrame,text="Face Recognition",font=("Courier", 60),bg="white").grid(row=1,column=1,columnspan=5)
loginButton = tk.Button(loginFrame,text="Login",bg="white",font=("Arial", 30),command=login)
loginButton.grid(row=2,column=5)
regButton = tk.Button(loginFrame,text="Register",command=regFrameRaiseFrame,bg="white",font=("Arial", 30))
regButton.grid(row=2,column=1)

tk.Label(regFrame,text="Register",font=("Courier",60),bg="white").grid(row=1,column=1,columnspan=5)
tk.Label(regFrame,text="Name: ",font=("Arial",30),bg="white").grid(row=2,column=1)
nameEntry=tk.Entry(regFrame,textvariable=name,font=("Arial",30)).grid(row=2,column=2)

registerButton = tk.Button(regFrame,text="Register",command=register,bg="white",font=("Arial", 30))
registerButton.grid(row=3,column=2)

tk.Label(userMenuFrame,text="Hello, ",font=("Courier",60),bg="white").grid(row=1,column=1)
tk.Label(userMenuFrame,textvariable=loggedInUser,font=("Courier",60),bg="white",fg="red").grid(row=1,column=2)
tk.Button(userMenuFrame,text="Back",font=("Arial", 30),command=logFrameRaiseFrame).grid(row=2,column=1)

